# 读取 .npy 数据、重现采样网格、提取切片、重塑为二维矩阵并绘制三维曲面图

- 数据读取：Xvars_Sweep.npy（设计变量）与 Result_Sweep.npy（7 维指标）。
- 关键指标：临界速度 critical velocity、磨耗数 wear number（刚性轮对、独立轮对 IRW）、Sperling 指数 Sperling index。
- 图形：与 MATLAB surf 一致的 3D 曲面，并可选保存为 PNG（-r600 等效）。
- 兼容中文显示：自动检测操作系统并设置 Matplotlib 字体回退。

注：MATLAB 的 reshape 为列主序（Fortran 顺序）。为与 MATLAB 结果逐点一致，实现中使用 order='F' 的 numpy.reshape。

In [ ]:
# 导入与中文字体设置
import os
from pathlib import Path
import platform
import warnings
from itertools import count

import sys, subprocess
from IPython import get_ipython

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (激活 3D 投影)
from matplotlib import font_manager as fm


# 切换后端为 widget（无需重启内核）
ip = get_ipython()
if ip is not None:
    ip.run_line_magic("matplotlib", "widget")
else:
    print("[Warn] 当前环境非 IPython；交互后端可能无法启用。")

print("[Info] Matplotlib backend ->", matplotlib.get_backend())
print("[Hint] 现在重新运行绘图 cell（例如 {Lx1,Lx2}->CriticalVel 的那一格），即可用鼠标旋转。")

# ---------- 中文字体与负号显示 ----------
def _pick_first_installed(candidates):
    """在已安装字体中，返回第一个可用的中文字体名称。"""
    installed = {f.name for f in fm.fontManager.ttflist}
    for name in candidates:
        if name in installed:
            return name
    return None

def setup_matplotlib_chinese():
    sys_name = platform.system()
    if sys_name == "Windows":
        candidates = ["Microsoft YaHei", "SimHei"]
    elif sys_name == "Darwin":  # macOS
        candidates = ["PingFang SC", "Heiti SC", "Songti SC"]
    else:  # Linux/Other
        candidates = ["Noto Sans CJK SC", "WenQuanYi Zen Hei", "AR PL UMing CN"]
    chosen = _pick_first_installed(candidates) or "DejaVu Sans"
    matplotlib.rcParams["font.sans-serif"] = [chosen, "DejaVu Sans"]
    matplotlib.rcParams["axes.unicode_minus"] = False
    return sys_name, chosen

sys_name, chosen_font = setup_matplotlib_chinese()
print(f"[Info] OS: {sys_name}, 使用中文字体: {chosen_font}")

# ---------- 工具函数 ----------
FIG_NUM = count(start=1)
def next_fig_label():
    """返回 '图1'、'图2' ... 的编号标签。"""
    return f"图{next(FIG_NUM)}"

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def fmt_float_for_fname(x: float) -> str:
    """将浮点数转为文件名友好格式，如 0.6 -> '0p6', -0.05 -> 'm0p05'。"""
    s = f"{x:.6g}"
    return s.replace("-", "m").replace(".", "p")

# 统一图像分辨率（与 MATLAB print -r600 等效的保存分辨率）
SAVE_DPI = 600
DISPLAY_DPI = 120  # notebook 预览用


# 数据与输出路径

请在下方**仅修改** `DATA_DIR` 与 `OUTPUT_DIR` 为本地的目录。其余变量/逻辑与 MATLAB 脚本一致。

In [ ]:
# 设置数据与输出目录
# <<< 请按需修改 >>>
DATA_DIR = Path(r"F:\ResearchMainStream\0.ResearchBySection\C.动力学模型\C23参数优化\参数优化实现\ParallelSweepSimpack\结果分析组\多指标参数扫略评价\真正刚性轮对用于计算-0227")
OUTPUT_DIR = Path.cwd() / "sweep_plots_output"  # 结果储存目录
ensure_dir(OUTPUT_DIR)

# 文件名（保持与 MATLAB 一致）
XVAR_FILE = "Xvars_Sweep.npy"
RLT_FILE  = "Result_Sweep.npy"

xvar_path = DATA_DIR / XVAR_FILE
rlt_path  = DATA_DIR / RLT_FILE

print("[Info] DATA_DIR =", DATA_DIR)
print("[Info] OUTPUT_DIR =", OUTPUT_DIR)
print("[Check] XVAR exists:", xvar_path.exists())
print("[Check] RLT  exists:", rlt_path.exists())


# 数据加载（Load .npy）
读取 `.npy` 数据并检查尺寸

- `Xvar_L123`：设计变量矩阵，按 MATLAB 脚本使用的行索引：
  - Lx1 → 第 30 行（MATLAB 1-based），Python 对应 `index=29`
  - Lx2 → 第 31 行（MATLAB 1-based），Python 对应 `index=30`
  - Lx3 → 第 32 行（MATLAB 1-based），Python 对应 `index=31`
- `Rlt_7dims`：7 个指标（行）× 组合数（列），将用到：
  - 行0：临界速度 *critical velocity*（m/s）
  - 行1：刚性轮对总磨耗数 *total wear number (rigid)*
  - 行3：IRW（独立轮对 *independently rotating wheelset*）总磨耗数
  - 行5：Sperling Y
  - 行6：Sperling Z

In [ ]:
# 加载数据与基本信息
Xvar_L123 = np.load(xvar_path)  # shape: (vars, N)
Rlt_7dims = np.load(rlt_path)   # shape: (7, N)

print("Xvar_L123.shape =", Xvar_L123.shape)
print("Rlt_7dims.shape =", Rlt_7dims.shape)

# 数据类型与基本统计预览
print("Xvar_L123 dtype:", Xvar_L123.dtype, "| Rlt_7dims dtype:", Rlt_7dims.dtype)
N_cols = Xvar_L123.shape[1]
assert Rlt_7dims.shape[1] == N_cols, "Xvar 与 Rlt 列数（组合数）应一致。"

# 显示部分行（变量）与前几列，辅助理解
np.set_printoptions(precision=4, suppress=True)
print("Lx1 (row 29) sample:", Xvar_L123[29, :10])
print("Lx2 (row 30) sample:", Xvar_L123[30, :10])
print("Lx3 (row 31) sample:", Xvar_L123[31, :10])

# 采样网格重现（Reproduce Sweep Grids）
重现 Lx1/Lx2/Lx3 采样点与*维度匹配*检查

- Lx1: `0 : 0.04 : 0.64`（17 点）
- Lx2: `0 : 0.04 : 0.60`（16 点）
- Lx3: 缺省为单点 `0.0`（如需 12 点，取消注释对应段落）
- *容忍度 tolerance*：`tol = 1e-10`，用于浮点比较（等价于 MATLAB 中的“接近 0/给定值”判定）。

> 注：如果将 Lx3 扩展为 12 点，对应总体组合数为 `17*16*12=3264`。若 `.npy` 含完整 3264 组合而此处仅设 Lx3=0，则**全量维度检查会告警**，但后续切片（按 Lx3≈0）依然可正确运行。


In [ ]:
# 定义 sweep 网格与维度检查
Lx1_sweep = np.arange(0.0, 0.64 + 1e-12, 0.04)  # 17
Lx2_sweep = np.arange(0.0, 0.60 + 1e-12, 0.04)  # 16

# 方案A：单点 Lx3=0
Lx3_sweep = np.array([0.0])  # 1

# 方案B：如需完整 12 点，取消注释
# Lx3_sweep = np.arange(-0.6, 0.5 + 1e-12, 0.1)  # 12

nLx1, nLx2, nLx3 = len(Lx1_sweep), len(Lx2_sweep), len(Lx3_sweep)
print(f"nLx1={nLx1}, nLx2={nLx2}, nLx3={nLx3}")

expected_all = nLx1 * nLx2 * nLx3
if expected_all != Xvar_L123.shape[1]:
    warnings.warn(
        f"[Warn] 维度可能不匹配：期望 {expected_all} 列，但实际 {Xvar_L123.shape[1]} 列。"
        " 将基于切片索引（如 Lx3≈0）继续运行，这与原 MATLAB 脚本在此情况下的意图一致。"
    )

tol = 1e-10  # 浮点容忍度 tolerance


# 切片索引（Indexing by Conditions）
计算关键切片索引

与 MATLAB 等价：
- `index_Lx3_00`: `Lx3 ≈ 0`
- `index_Lx1_06`: `Lx1 = 0.6`
- `index_Lx2_04`: `Lx2 = 0.4`


In [ ]:
# 计算索引
# MATLAB(1-based): L1=30, L2=31, L3=32 -> Python(0-based): 29, 30, 31
idx_Lx3_00 = np.where(np.abs(Xvar_L123[31, :] - 0.0) < tol)[0]
idx_Lx1_06 = np.where(np.abs(Xvar_L123[29, :] - 0.6) < tol)[0]
idx_Lx2_04 = np.where(np.abs(Xvar_L123[30, :] - 0.4) < tol)[0]

print("len(index_Lx3_00) =", len(idx_Lx3_00), "(应≈ nLx1*nLx2 =", nLx1 * nLx2, ")")
print("len(index_Lx1_06) =", len(idx_Lx1_06), "(应≈ nLx2*nLx3 =", nLx2 * nLx3, ")")
print("len(index_Lx2_04) =", len(idx_Lx2_04), "(应≈ nLx1*nLx3 =", nLx1 * nLx3, ")")

# 展示前若干索引，便于快速核对
print("index_Lx3_00[:10] ->", idx_Lx3_00[:10])

# 提取与重塑（Extract & Reshape）
- 提取指标并重塑为二维矩阵（与 MATLAB reshape 一致）
- 为保持与 MATLAB reshape（列主序 column-major）一致，这里使用：arr_2d = arr_1d.reshape((n_rows, n_cols), order="F")

对应关系：
- CriticalVel_2D_L1L2：从 Lx3≈0 切片得到，形状 (nLx2, nLx1)，并转换 km/h（*3.6）。
- 同理生成 CriticalVel_2D_L1L3、CriticalVel_2D_L2L3（用于需要时的可视化）。
- 同步提取磨耗与 Sperling 相关二维矩阵。

In [ ]:
# 提取与 reshape（Fortran 顺序以匹配 MATLAB）
# --- Lx3 ≈ 0 切片 ---
CriticalVel_Lx3Eq00 = Rlt_7dims[0, idx_Lx3_00]  # m/s
WearN_rigid_Lx3Eq00 = Rlt_7dims[1, idx_Lx3_00]
WearNum_IRW_Lx3Eq00 = Rlt_7dims[3, idx_Lx3_00]

# --- Lx1 = 0.6 切片 ---
CriticalVel_Lx1Eq06 = Rlt_7dims[0, idx_Lx1_06]  # m/s

# --- Lx2 = 0.4 切片 ---
CriticalVel_Lx2Eq04 = Rlt_7dims[0, idx_Lx2_04]  # m/s

# 2D 重塑（列主序）
CriticalVel_2D_L1L2 = np.reshape(CriticalVel_Lx3Eq00, (nLx2, nLx1), order="F") * 3.6  # km/h
CriticalVel_2D_L1L3 = np.reshape(CriticalVel_Lx2Eq04, (nLx3, nLx1), order="F") * 3.6  # km/h
CriticalVel_2D_L2L3 = np.reshape(CriticalVel_Lx1Eq06, (nLx3, nLx2), order="F") * 3.6  # km/h

# 其他指标（用于函数内绘图）
WearRigid_2D_L1L2 = np.reshape(WearN_rigid_Lx3Eq00, (nLx2, nLx1), order="F")
WearIRW_2D_L1L2   = np.reshape(WearNum_IRW_Lx3Eq00, (nLx2, nLx1), order="F")

print("CriticalVel_2D_L1L2.shape =", CriticalVel_2D_L1L2.shape)
print("WearRigid_2D_L1L2.shape   =", WearRigid_2D_L1L2.shape)
print("WearIRW_2D_L1L2.shape     =", WearIRW_2D_L1L2.shape)

# 基本统计（km/h）
def stats(a):
    return {"min": float(np.min(a)), "max": float(np.max(a)), "mean": float(np.mean(a))}
print("CriticalVel_2D_L1L2 stats (km/h):", stats(CriticalVel_2D_L1L2))


# 三维可视化 - A（{Lx1, Lx2} → 临界速度，Lx3=0）
生成三维曲面图（与 MATLAB `surf` 对齐）

- 横轴：Lx1（双拉杆构架侧间距）
- 纵轴：Lx2（双拉杆轴桥侧间距）
- 竖轴：Critical Velocity (km/h)
- 限制范围与标题与 MATLAB 一致，并显示颜色条 *colorbar*。
- **保存逻辑**：由 SAVE_MODE 控制（"ToPrint" 保存 PNG；"NotPrint" 仅显示）。

In [ ]:
# 绘制 {Lx1, Lx2} -> 临界速度 曲面（依赖：CriticalVel_2D_L1L2, Lx1_sweep, Lx2_sweep）
SAVE_MODE = "NotPrint"  # 可改为 "ToPrint" / NotPrint
X, Y = np.meshgrid(Lx1_sweep, Lx2_sweep, indexing="xy")
Z = CriticalVel_2D_L1L2  # km/h

fig = plt.figure(figsize=(12, 8), dpi=DISPLAY_DPI)
ax = fig.add_subplot(111, projection="3d")
surf = ax.plot_surface(X, Y, Z, linewidth=0, antialiased=True, cmap="viridis")
ax.set_xlabel("双拉杆构架侧间距 Lx1")
ax.set_ylabel("双拉杆轴桥侧间距 Lx2")
ax.set_zlabel("Critical Velocity (km/h)")
ax.set_xlim(0.0, 0.6)
ax.set_ylim(0.0, 0.6)
ax.set_zlim(300.0, 500.0)
fig.colorbar(surf, ax=ax, shrink=0.7, pad=0.1)

title = f"{next_fig_label()} - Lx1、Lx2 对于临界速度的影响，Lx3=0"
ax.set_title(title)
plt.show()

if SAVE_MODE == "ToPrint":
    out_path = OUTPUT_DIR / f"CriticalVel_L1L2_Lx3is0.png"
    fig.savefig(out_path, dpi=SAVE_DPI, bbox_inches="tight")
    print(f"[Saved] {out_path}")


# 定义：7维度指标作图函数（等价于 MATLAB 局部函数）
plot_7dims_from_3264 

功能：
- 给定 Lx3 值，按 abs(Lx3实际 - Lx3目标) < tol 取列索引。
- 将 7 维指标的相关行重塑为 (nLx2, nLx1) 矩阵（Fortran 顺序）。
- 生成三张图：
  1) 临界速度（单位转为 km/h）
  2) 磨耗数（刚性轮对 / 独立轮对 IRW）
  3) Sperling 指数（Y / Z）
- 标注轴范围、标题与颜色条；按 isprint 决定是否保存至 save_dir。

指标行与 MATLAB 一致（Python 0-based）：
- `Rlt_7dims[0,:]` → 临界速度（m/s）
- `Rlt_7dims[1,:]` → 刚性轮对总磨耗数
- `Rlt_7dims[3,:]` → IRW 总磨耗数
- `Rlt_7dims[5,:]` → Sperling Y
- `Rlt_7dims[6,:]` → Sperling Z

In [ ]:
# 定义函数 plot_7dims_from_3264
def plot_7dims_from_3264(
    Lx3_value: float,
    Xvar_L123: np.ndarray,
    Rlt_7dims: np.ndarray,
    Lx1_sweep: np.ndarray,
    Lx2_sweep: np.ndarray,
    nLx1: int,
    nLx2: int,
    L3Id: int,
    save_dir: Path,
    isprint: str = "NotPrint",
    tol: float = 1e-10,
):
    """
    等价于 MATLAB 的 Plot7dimsFrom3264：
    - 根据 Lx3（目标值）筛选列
    - 重塑为 (nLx2, nLx1) 并绘制三张图（含 colorbar）
    - isprint: "ToPrint" 将保存 3 张 PNG；"NotPrint" 仅显示
    """
    ensure_dir(save_dir)

    # 依据 Lx3 接近判断取索引（MATLAB 1-based 的 row 32 -> Python index 31）
    idx_Lx3 = np.where(np.abs(Xvar_L123[31, :] - Lx3_value) < tol)[0]
    if len(idx_Lx3) != nLx1 * nLx2:
        warnings.warn(
            f"[Warn] Lx3={Lx3_value} 切片列数={len(idx_Lx3)}，"
            f"与期望 nLx1*nLx2={nLx1*nLx2} 不符。将尝试继续绘图。"
        )

    # 切片后的 7 维指标
    Rlt_DDR = Rlt_7dims[:, idx_Lx3]

    # ---------- 临界速度（km/h） ----------
    CriticalVel_2D = np.reshape(Rlt_DDR[0, :], (nLx2, nLx1), order="F") * 3.6

    fig_critical = plt.figure(figsize=(12, 8), dpi=DISPLAY_DPI)
    ax1 = fig_critical.add_subplot(111, projection="3d")
    X, Y = np.meshgrid(Lx1_sweep, Lx2_sweep, indexing="xy")
    surf1 = ax1.plot_surface(X, Y, CriticalVel_2D, linewidth=0, antialiased=True, cmap="viridis")
    ax1.set_xlabel("双拉杆构架侧间距 Lx1")
    ax1.set_ylabel("双拉杆轴桥侧间距 Lx2")
    ax1.set_zlabel("Critical Velocity (km/h)")
    ax1.set_xlim(0.0, 0.6)
    ax1.set_ylim(0.0, 0.6)
    ax1.set_zlim(0.0, 700.0)
    fig_critical.colorbar(surf1, ax=ax1, shrink=0.7, pad=0.1)
    ax1.set_title(f"{next_fig_label()} - Lx1、Lx2 对于临界速度的影响, Lx3={Lx3_value:g}")
    plt.show()

    if isprint == "ToPrint":
        p = save_dir / f"CriticalVel_Fig{L3Id}_Lx3is{fmt_float_for_fname(Lx3_value)}.png"
        fig_critical.savefig(p, dpi=SAVE_DPI, bbox_inches="tight")
        print(f"[Saved] {p}")

    # ---------- 磨耗数（刚性/IRW） ----------
    RigidSumWear_2D = np.reshape(Rlt_DDR[1, :], (nLx2, nLx1), order="F")
    IRWSumWear_2D   = np.reshape(Rlt_DDR[3, :], (nLx2, nLx1), order="F")

    fig_wear = plt.figure(figsize=(10, 15), dpi=DISPLAY_DPI)

    ax2 = fig_wear.add_subplot(2, 1, 1, projection="3d")
    surf2 = ax2.plot_surface(X, Y, RigidSumWear_2D, linewidth=0, antialiased=True, cmap="viridis")
    ax2.set_xlabel("双拉杆构架侧间距 Lx1")
    ax2.set_ylabel("双拉杆轴桥侧间距 Lx2")
    ax2.set_zlabel("Total Wear Number (N)")
    ax2.set_xlim(0.0, 0.6)
    ax2.set_ylim(0.0, 0.6)
    ax2.set_zlim(0.0, 3000.0)
    fig_wear.colorbar(surf2, ax=ax2, shrink=0.7, pad=0.1)
    ax2.set_title(f"Lx1、Lx2 对于总磨耗数的影响（刚性轮对）, Lx3={Lx3_value:g}")

    ax3 = fig_wear.add_subplot(2, 1, 2, projection="3d")
    surf3 = ax3.plot_surface(X, Y, IRWSumWear_2D, linewidth=0, antialiased=True, cmap="viridis")
    ax3.set_xlabel("双拉杆构架侧间距 Lx1")
    ax3.set_ylabel("双拉杆轴桥侧间距 Lx2")
    ax3.set_zlabel("Total Wear Number (N)")
    ax3.set_xlim(0.0, 0.6)
    ax3.set_ylim(0.0, 0.6)
    ax3.set_zlim(0.0, 1000.0)
    fig_wear.colorbar(surf3, ax=ax3, shrink=0.7, pad=0.1)
    ax3.set_title(f"Lx1、Lx2 对于总磨耗数的影响（独立轮对 IRW）, Lx3={Lx3_value:g}")

    # 为整张图添加统一编号标题
    fig_wear.suptitle(next_fig_label() + " - 磨耗数指标（刚性/IRW）", y=0.92)
    plt.tight_layout()
    plt.show()

    if isprint == "ToPrint":
        p = save_dir / f"WearNumber_Fig{L3Id}_Lx3is{fmt_float_for_fname(Lx3_value)}.png"
        fig_wear.savefig(p, dpi=SAVE_DPI, bbox_inches="tight")
        print(f"[Saved] {p}")

    # ---------- Sperling 指标（Y/Z） ----------
    SperlingY_2D = np.reshape(Rlt_DDR[5, :], (nLx2, nLx1), order="F")
    SperlingZ_2D = np.reshape(Rlt_DDR[6, :], (nLx2, nLx1), order="F")

    fig_sp = plt.figure(figsize=(10, 15), dpi=DISPLAY_DPI)

    ax4 = fig_sp.add_subplot(2, 1, 1, projection="3d")
    surf4 = ax4.plot_surface(X, Y, SperlingY_2D, linewidth=0, antialiased=True, cmap="viridis")
    ax4.set_xlabel("双拉杆构架侧间距 Lx1")
    ax4.set_ylabel("双拉杆轴桥侧间距 Lx2")
    ax4.set_zlabel("Sperling Y")
    ax4.set_xlim(0.0, 0.6)
    ax4.set_ylim(0.0, 0.6)
    ax4.set_zlim(2.5, 3.0)
    fig_sp.colorbar(surf4, ax=ax4, shrink=0.7, pad=0.1)
    ax4.set_title(f"Lx1、Lx2 对于 Sperling Y 的影响, Lx3={Lx3_value:g}")

    ax5 = fig_sp.add_subplot(2, 1, 2, projection="3d")
    surf5 = ax5.plot_surface(X, Y, SperlingZ_2D, linewidth=0, antialiased=True, cmap="viridis")
    ax5.set_xlabel("双拉杆构架侧间距 Lx1")
    ax5.set_ylabel("双拉杆轴桥侧间距 Lx2")
    ax5.set_zlabel("Sperling Z")
    ax5.set_xlim(0.0, 0.6)
    ax5.set_ylim(0.0, 0.6)
    ax5.set_zlim(2.15, 2.3)
    fig_sp.colorbar(surf5, ax=ax5, shrink=0.7, pad=0.1)
    ax5.set_title(f"Lx1、Lx2 对于 Sperling Z 的影响, Lx3={Lx3_value:g}")

    fig_sp.suptitle(next_fig_label() + " - Sperling 指标（Y/Z）", y=0.92)
    plt.tight_layout()
    plt.show()

    if isprint == "ToPrint":
        p = save_dir / f"Sperling_Fig{L3Id}_Lx3is{fmt_float_for_fname(Lx3_value)}.png"
        fig_sp.savefig(p, dpi=SAVE_DPI, bbox_inches="tight")
        print(f"[Saved] {p}")


# 调用：批量生成 7 指标图组
依据 Lx3 列表循环绘图与（可选）保存

- `SAVE_MODE` 统一控制保存/仅显示。
- `SAVE_SUBDIR` 对应 MATLAB 的 `SavePath`（可使用中文目录名）。

In [ ]:
# 循环调用（依赖：plot_7dims_from_3264, Lx3_sweep, isprint, 目录）
isprint = "NotPrint" if SAVE_MODE != "ToPrint" else "ToPrint"
SAVE_SUBDIR = OUTPUT_DIR / "扫略L123获得7指标分析结果图组"
ensure_dir(SAVE_SUBDIR)

for L3Id, L3_val in enumerate(Lx3_sweep, start=1):
    print(f"[Run] L3Id={L3Id}, Lx3={L3_val}")
    plot_7dims_from_3264(
        Lx3_value=L3_val,
        Xvar_L123=Xvar_L123,
        Rlt_7dims=Rlt_7dims,
        Lx1_sweep=Lx1_sweep,
        Lx2_sweep=Lx2_sweep,
        nLx1=nLx1,
        nLx2=nLx2,
        L3Id=L3Id,
        save_dir=SAVE_SUBDIR,
        isprint=isprint,
        tol=tol,
    )

print("[Done] 全部绘图完成。保存模式:", isprint)
